# Cellpose Installation and Testing in MyBinder

This notebook demonstrates how to set up Cellpose in a MyBinder environment using miniforge3 and test that it works correctly without a GUI.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/your-username/your-repo/HEAD?filepath=cellpose_test.ipynb)

## Overview
- Install miniforge3 and create virtual environment
- Install Cellpose and dependencies
- Test installation with sample data
- Demonstrate basic usage without GUI

## Step 1: Environment Setup

First, let's check our current environment and set up miniforge3:

In [ ]:
import sys
import os
import subprocess

print(f"Python version: {sys.version}")
print(f"Current working directory: {os.getcwd()}")
print(f"Python executable: {sys.executable}")

In [ ]:
# Check if conda/mamba is available
try:
    result = subprocess.run(['mamba', '--version'], capture_output=True, text=True)
    print("Mamba version:", result.stdout)
    CONDA_CMD = 'mamba'
except FileNotFoundError:
    try:
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        print("Conda version:", result.stdout)
        CONDA_CMD = 'conda'
    except FileNotFoundError:
        print("Neither mamba nor conda found. Installing miniforge3...")
        CONDA_CMD = None

## Step 2: Install Miniforge3 (if needed)

If conda/mamba isn't available, we'll install miniforge3:

In [ ]:
if CONDA_CMD is None:
    # Download and install miniforge3
    !wget https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh -O miniforge3.sh
    !bash miniforge3.sh -b -p $HOME/miniforge3
    
    # Add to PATH
    os.environ['PATH'] = f"{os.path.expanduser('~/miniforge3/bin')}:{os.environ['PATH']}"
    
    # Verify installation
    result = subprocess.run([f"{os.path.expanduser('~/miniforge3/bin/mamba')}", '--version'], 
                          capture_output=True, text=True)
    print("Mamba installed:", result.stdout)
    CONDA_CMD = f"{os.path.expanduser('~/miniforge3/bin/mamba')}"
else:
    print(f"Using existing {CONDA_CMD}")

## Step 3: Create Virtual Environment for Cellpose

Now let's create a dedicated environment for Cellpose:

In [ ]:
# Create new environment with Python 3.9 (recommended for Cellpose)
env_name = "cellpose_env"

print(f"Creating environment: {env_name}")
!{CONDA_CMD} create -n {env_name} python=3.9 -y

print(f"Environment {env_name} created successfully!")

## Step 4: Install Cellpose and Dependencies

Install Cellpose and required packages in our virtual environment:

In [ ]:
# Install cellpose and common dependencies
print("Installing Cellpose and dependencies...")

# Install via conda-forge first for better compatibility
!{CONDA_CMD} install -n {env_name} -c conda-forge numpy scipy matplotlib scikit-image opencv tifffile -y

# Install cellpose via pip in the environment
conda_prefix = subprocess.run([CONDA_CMD, 'info', '--envs'], capture_output=True, text=True).stdout
env_path = None
for line in conda_prefix.split('\n'):
    if env_name in line:
        env_path = line.split()[-1]
        break

if env_path:
    pip_path = f"{env_path}/bin/pip"
    print(f"Installing cellpose using pip at: {pip_path}")
    !{pip_path} install cellpose[all]
else:
    print("Could not find environment path, using conda run instead")
    !{CONDA_CMD} run -n {env_name} pip install cellpose[all]

print("Installation complete!")

## Step 5: Verify Installation

Let's test that Cellpose was installed correctly:

In [ ]:
# Test import in the cellpose environment
test_script = """
try:
    import cellpose
    from cellpose import models, io, utils
    import numpy as np
    import matplotlib.pyplot as plt
    
    print(f"✅ Cellpose version: {cellpose.__version__}")
    print("✅ All imports successful!")
    
    # Test model loading
    model = models.Cellpose(gpu=False, model_type='cyto')
    print("✅ Model loaded successfully!")
    
    # Check available models
    print(f"Available models: {models.MODEL_NAMES}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
"""

# Run test in cellpose environment
with open('test_cellpose.py', 'w') as f:
    f.write(test_script)

!{CONDA_CMD} run -n {env_name} python test_cellpose.py

## Step 6: Create Sample Data for Testing

Let's create some synthetic cell-like data to test Cellpose:

In [ ]:
# Create synthetic test data
sample_data_script = """
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import morphology

# Create synthetic cell-like image
def create_synthetic_cells(size=(512, 512), n_cells=20):
    img = np.zeros(size)
    
    # Add random circular objects (cells)
    for i in range(n_cells):
        # Random center
        center_y = np.random.randint(50, size[0]-50)
        center_x = np.random.randint(50, size[1]-50)
        
        # Random radius
        radius = np.random.randint(15, 35)
        
        # Create circular mask
        y, x = np.ogrid[:size[0], :size[1]]
        mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
        
        # Add cell with some intensity variation
        intensity = np.random.uniform(150, 255)
        img[mask] = intensity
    
    # Add some noise
    noise = np.random.normal(0, 10, size)
    img = img + noise
    img = np.clip(img, 0, 255)
    
    # Smooth slightly
    img = ndimage.gaussian_filter(img, sigma=1)
    
    return img.astype(np.uint8)

# Generate test image
test_img = create_synthetic_cells()

# Save the test image
import tifffile
tifffile.imwrite('test_cells.tif', test_img)

# Display the test image
plt.figure(figsize=(8, 8))
plt.imshow(test_img, cmap='gray')
plt.title('Synthetic Test Cells')
plt.axis('off')
plt.savefig('test_cells_preview.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Test image shape: {test_img.shape}")
print(f"Test image dtype: {test_img.dtype}")
print(f"Intensity range: {test_img.min()} - {test_img.max()}")
"""

with open('create_test_data.py', 'w') as f:
    f.write(sample_data_script)

!{CONDA_CMD} run -n {env_name} python create_test_data.py

## Step 7: Run Cellpose Segmentation Test

Now let's test Cellpose segmentation on our synthetic data:

In [ ]:
# Cellpose segmentation test
segmentation_script = """
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io
import tifffile
import time

print("Loading test image...")
img = tifffile.imread('test_cells.tif')
print(f"Image loaded: {img.shape}")

# Initialize Cellpose model
print("Initializing Cellpose model...")
model = models.Cellpose(gpu=False, model_type='cyto')
print("Model initialized!")

# Run segmentation
print("Running segmentation...")
start_time = time.time()

masks, flows, styles, diams = model.eval(img, diameter=None, channels=[0,0])

end_time = time.time()
print(f"Segmentation completed in {end_time - start_time:.2f} seconds")

# Analyze results
n_cells = len(np.unique(masks)) - 1  # subtract 1 for background
print(f"Number of cells detected: {n_cells}")
print(f"Estimated diameter: {diams:.1f} pixels")

# Create visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Original image
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# Segmentation masks
axes[1].imshow(masks, cmap='nipy_spectral')
axes[1].set_title(f'Segmentation Masks\n({n_cells} cells detected)')
axes[1].axis('off')

# Overlay
from cellpose import utils
overlay = utils.masks_to_outlines(masks)
axes[2].imshow(img, cmap='gray', alpha=0.7)
axes[2].imshow(overlay, cmap='red', alpha=0.3)
axes[2].set_title('Overlay')
axes[2].axis('off')

plt.tight_layout()
plt.savefig('cellpose_results.png', dpi=150, bbox_inches='tight')
plt.show()

# Save results
tifffile.imwrite('segmentation_masks.tif', masks.astype(np.uint16))
print("Results saved to 'segmentation_masks.tif'")

print("\n✅ Cellpose segmentation test completed successfully!")
"""

with open('test_segmentation.py', 'w') as f:
    f.write(segmentation_script)

!{CONDA_CMD} run -n {env_name} python test_segmentation.py

## Step 8: Test Different Models and Parameters

Let's test different Cellpose models to show the flexibility:

In [ ]:
# Test multiple models
model_test_script = """
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
import tifffile

# Load test image
img = tifffile.imread('test_cells.tif')

# Test different models
model_types = ['cyto', 'nuclei', 'cyto2']
results = {}

for model_type in model_types:
    try:
        print(f"Testing {model_type} model...")
        model = models.Cellpose(gpu=False, model_type=model_type)
        masks, flows, styles, diams = model.eval(img, diameter=None, channels=[0,0])
        n_cells = len(np.unique(masks)) - 1
        results[model_type] = {
            'masks': masks,
            'n_cells': n_cells,
            'diameter': diams
        }
        print(f"  - {n_cells} cells detected")
        print(f"  - Estimated diameter: {diams:.1f} pixels")
    except Exception as e:
        print(f"  - Error with {model_type}: {e}")

# Visualize comparison
fig, axes = plt.subplots(2, len(results), figsize=(5*len(results), 10))
if len(results) == 1:
    axes = axes.reshape(-1, 1)

for i, (model_type, result) in enumerate(results.items()):
    # Original image
    axes[0, i].imshow(img, cmap='gray')
    axes[0, i].set_title(f'Original Image\n({model_type} model)')
    axes[0, i].axis('off')
    
    # Segmentation
    axes[1, i].imshow(result['masks'], cmap='nipy_spectral')
    axes[1, i].set_title(f'{model_type.upper()} Model\n{result["n_cells"]} cells')
    axes[1, i].axis('off')

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Model Comparison Summary:")
for model_type, result in results.items():
    print(f"  {model_type.upper()}: {result['n_cells']} cells (avg diameter: {result['diameter']:.1f}px)")
"""

with open('test_models.py', 'w') as f:
    f.write(model_test_script)

!{CONDA_CMD} run -n {env_name} python test_models.py

## Step 9: Performance and Environment Summary

Let's gather some final information about our setup:

In [ ]:
# Environment summary
summary_script = """
import sys
import os
import cellpose
import numpy as np
import torch
from cellpose import models

print("🔍 ENVIRONMENT SUMMARY")
print("=" * 50)
print(f"Python version: {sys.version.split()[0]}")
print(f"Cellpose version: {cellpose.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Environment: {os.environ.get('CONDA_DEFAULT_ENV', 'Unknown')}")

print("\n📋 AVAILABLE MODELS")
print("=" * 50)
for model_name in models.MODEL_NAMES:
    print(f"  ✓ {model_name}")

print("\n💾 GENERATED FILES")
print("=" * 50)
files = ['test_cells.tif', 'test_cells_preview.png', 'cellpose_results.png', 
         'segmentation_masks.tif', 'model_comparison.png']
for file in files:
    if os.path.exists(file):
        size = os.path.getsize(file) / 1024  # KB
        print(f"  ✓ {file} ({size:.1f} KB)")
    else:
        print(f"  ✗ {file} (not found)")

print("\n🎉 CELLPOSE SETUP COMPLETE!")
print("=" * 50)
print("Your Cellpose installation is working correctly in MyBinder.")
print("You can now use Cellpose for cell segmentation without needing a GUI!")
"""

with open('environment_summary.py', 'w') as f:
    f.write(summary_script)

!{CONDA_CMD} run -n {env_name} python environment_summary.py

## Step 10: Usage Examples for Your Research

Here are some practical examples of how to use Cellpose programmatically:

In [ ]:
# Practical usage examples
usage_examples_script = """
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io, utils
import tifffile

print("📚 PRACTICAL CELLPOSE USAGE EXAMPLES")
print("=" * 60)

# Load your test image
img = tifffile.imread('test_cells.tif')

# Example 1: Batch processing with custom parameters
print("\n1️⃣  Batch Processing Example")
model = models.Cellpose(gpu=False, model_type='cyto')

# Custom parameters
custom_params = {
    'diameter': 30,        # Expected cell diameter
    'flow_threshold': 0.4, # Flow error threshold
    'cellprob_threshold': 0.0  # Cell probability threshold
}

masks, flows, styles, diams = model.eval(
    img, 
    channels=[0,0],  # Grayscale
    **custom_params
)

print(f"   Custom parameters result: {len(np.unique(masks))-1} cells")

# Example 2: Extract cell properties
print("\n2️⃣  Cell Property Analysis")
from skimage import measure

props = measure.regionprops(masks, img)
areas = [prop.area for prop in props]
intensities = [prop.mean_intensity for prop in props]

print(f"   Average cell area: {np.mean(areas):.1f} ± {np.std(areas):.1f} pixels²")
print(f"   Average intensity: {np.mean(intensities):.1f} ± {np.std(intensities):.1f}")
print(f"   Size range: {min(areas):.0f} - {max(areas):.0f} pixels²")

# Example 3: Quality control metrics
print("\n3️⃣  Quality Control")
n_cells = len(props)
image_area = img.shape[0] * img.shape[1]
total_cell_area = sum(areas)
coverage = (total_cell_area / image_area) * 100

print(f"   Cell density: {n_cells / (image_area/10000):.2f} cells per 10,000 pixels²")
print(f"   Coverage: {coverage:.1f}% of image")
print(f"   Flow threshold used: {custom_params['flow_threshold']}")

# Example 4: Save results in different formats
print("\n4️⃣  Export Options")

# Save as labeled TIFF
tifffile.imwrite('labeled_cells.tif', masks.astype(np.uint16))

# Save outlines as binary mask
outlines = utils.masks_to_outlines(masks)
tifffile.imwrite('cell_outlines.tif', outlines.astype(np.uint8) * 255)

# Save data as CSV
import pandas as pd
cell_data = pd.DataFrame({
    'cell_id': range(1, len(props) + 1),
    'area': areas,
    'mean_intensity': intensities,
    'centroid_x': [prop.centroid[1] for prop in props],
    'centroid_y': [prop.centroid[0] for prop in props]
})
cell_data.to_csv('cell_measurements.csv', index=False)

print("   ✓ labeled_cells.tif (16-bit labeled image)")
print("   ✓ cell_outlines.tif (binary outlines)")
print("   ✓ cell_measurements.csv (quantitative data)")

print("\n🎯 READY FOR YOUR RESEARCH!")
print("=" * 60)
print("Your Cellpose environment is fully functional.")
print("You can now upload your own images and start segmenting!")
"""

with open('usage_examples.py', 'w') as f:
    f.write(usage_examples_script)

!{CONDA_CMD} run -n {env_name} python usage_examples.py

## 🎉 Success!

Congratulations! You now have a fully functional Cellpose installation in MyBinder that:

✅ **Works without GUI** - Perfect for programmatic usage  
✅ **Uses virtual environment** - Clean, isolated installation  
✅ **Includes all models** - cyto, nuclei, cyto2, and more  
✅ **Provides examples** - Ready-to-use code snippets  
✅ **Exports results** - Multiple output formats  

### Next Steps:
1. **Upload your own images** to test with real data
2. **Modify parameters** in the examples above
3. **Scale up** for batch processing multiple images
4. **Integrate** with your existing analysis pipeline

### MyBinder Repository Setup:
To use this notebook in MyBinder, create a repository with:
- This notebook file (`cellpose_test.ipynb`)
- Optional: `environment.yml` file for faster setup
- Optional: `requirements.txt` for additional packages

**Happy cell segmenting! 🔬**